### Imports

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from keras.utils import to_categorical as make_class_categorical
import _pickle as pickle
from tqdm import tqdm
import pdb

Using TensorFlow backend.


### Assignment 2 functions

#### Initialize weights

In [2]:
def initialize_weights(d, m, K, std=0.001):
    """
    Initializes the weight and bias arrays for the 2 layers of the network

    :param d: Dimensionality of the input data
    :param m: Number of nodes in the first layer
    :param K: Number of different classes (K=10 for the CIFAR-10 dataset)
    :param variance (optional): The variance of the normal distribution that will be used for the initialization of the weights

    :return: Weights and bias arrays for the first and second layer of the neural network
    """

    np.random.seed(400)

    W1 = np.random.normal(0, std, size=(m, d))
    b1 = np.zeros(shape=(m, 1))

    W2 = np.random.normal(0, std, size=(K, m))
    b2 = np.zeros(shape=(K, 1))

    return W1, b1, W2, b2

#### Load Batch

In [3]:
def LoadBatch(filename):
    """
    Loads batch based on the given filename and produces the X, Y, and y arrays

    :param filename: Path of the file
    :return: X, Y and y arrays
    """

    # borrowed from https://www.cs.toronto.edu/~kriz/cifar.html
    def unpickle(file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict

    dictionary = unpickle(filename)

    # borrowed from https://stackoverflow.com/questions/16977385/extract-the-nth-key-in-a-python-dictionary?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
    def ix(dic, n):  # don't use dict as  a variable name
        try:
            return list(dic)[n]  # or sorted(dic)[n] if you want the keys to be sorted
        except IndexError:
            print('not enough keys')

    garbage = ix(dictionary, 1)
    y = dictionary[garbage]
    Y = np.transpose(make_class_categorical(y, 10))
    garbage = ix(dictionary, 2)
    X = np.transpose(dictionary[garbage]) / 255

    return X, Y, y

#### ReLU


In [4]:
def ReLU(x):
    """
    Rectified Linear Unit function

    :param x: Input to the function

    :return: Output of ReLU(x)
    """

    return np.maximum(x, 0)

#### Softmax

In [16]:
def softmax(X, theta=1.0, axis=None):

    # Softmax over numpy rows and columns, taking care for overflow cases
    # Many thanks to https://nolanbconaway.github.io/blog/2017/softmax-numpy
    # Usage: Softmax over rows-> axis =0, softmax over columns ->axis =1

    """
    Compute the softmax of each element along an axis of X.
    Parameters
    ----------
    X: ND-Array. Probably should be floats.
    theta (optional): float parameter, used as a multiplier
        prior to exponentiation. Default = 1.0
    axis (optional): axis to compute values along. Default is the
        first non-singleton axis.
    Returns an array the same size as X. The result will sum to 1
    along the specified axis.
    """

    # make X at least 2d
    y = np.atleast_2d(X)

    # find axis
    if axis is None:
        axis = next(j[0] for j in enumerate(y.shape) if j[1] > 1)

    # multiply y against the theta parameter,
    y = y * float(theta)

    # subtract the max for numerical stability
    y = y - np.expand_dims(np.max(y, axis=axis), axis)

    # exponentiate y
    y = np.exp(y)

    # take the sum along the specified axis
    ax_sum = np.expand_dims(np.sum(y, axis=axis), axis)

    # finally: divide elementwise
    p = y / ax_sum

    # flatten if X was 1D
    if len(X.shape) == 1: p = p.flatten()

    return p

#### Evaluate Classifier

In [5]:
def EvaluateClassifier(X, W1, b1, W2, b2):
    """
    Computes the Softmax output of the 2 layer network, based on input data X and trained weight and bias arrays

    :param X: Input data
    :param W1: Weight array of the first layer
    :param b1: Bias vector of the first layer
    :param W2: Weight array of the second layer
    :param b2: Bias vector of the second layer

    :return: Softmax output of the trained network
    """

    s1 = np.dot(W1, X) + b1
    h = ReLU(s1)
    s = np.dot(W2, h) + b2
    p = softmax(s, axis=0)

    return p, h, s1

#### Predict classes

In [6]:
def predictClasses(p):
    """
    Predicts classes based on the softmax output of the network

    :param p: Softmax output of the network
    :return: Predicted classes
    """

    return np.argmax(p, axis=0)

#### Compute Accuracy

In [7]:
def ComputeAccuracy(X, y, W1, b1, W2, b2):
    """
    Computes the accuracy of the feed-forward 2-layer network

    :param X: Input data
    :param y: Labels of the ground truth
    :param W1: Weight matrix of the first layer
    :param b1: Bias vector of the first layer
    :param W2: Weight matrix of the second layer
    :param b2: Bias vector of the second layer

    :return: Accuracy metric of the neural network.
    """
    p, _, _ = EvaluateClassifier(X=X, W1=W1, b1=b1, W2=W2, b2=b2)
    predictions = predictClasses(p)

    accuracy = np.sum(np.where(predictions - y == 0, 1, 0))

    return accuracy

#### Compute Cost

In [8]:
def ComputeCost(X, Y, W1, W2, b1, b2, regularization_term= 0):
    """
    Computes the cross-entropy loss on a batch of data.

    :param X: Input data
    :param y: Labels of the ground truth
    :param W1: Weight matrix of the first layer
    :param b1: Bias vector of the first layer
    :param W2: Weight matrix of the second layer
    :param b2: Bias vector of the second layer
    :param regularization_term: Amount of regularization applied.

    :return: Cross-entropy loss.
    """

    p, _, _ = EvaluateClassifier(X=X, W1=W1, b1=b1, W2=W2, b2=b2)

    cross_entropy_loss = np.sum(-np.log(np.dot(Y.T, p)), axis=1).sum() / float(X.shape[0])

    weight_sum = np.power(W1, 2).sum() + np.power(W2, 2).sum()

    return cross_entropy_loss + regularization_term * weight_sum

#### Compute Gradients

In [9]:
def ComputeGradsNum(X, Y, W1, b1, W2, b2, regularization_term, h=1e-5):

    """
    Computes gradient descent updates on a batch of data with numerical computations.
    Contributed by Josephine Sullivan for educational purposes for the DD2424 Deep Learning in Data Science course.

    :param X: Input data
    :param Y: One-hot representation of the true labels of input data X
    :param W1: Weight matrix of the first layer
    :param b1: Bias vector of the first layer
    :param W2: Weight matrix of the second layer
    :param b2: Bias vector of the second layer
    :param regularization_term: Contribution of the regularization in the weight updates

    :return: Weight and bias updates of the first and second layer of our network computed with numerical computations
    """

    grad_W1= np.zeros((W1.shape[0], W1.shape[1]))
    grad_b1= np.zeros((W1.shape[0], 1))
    grad_W2= np.zeros((W2.shape[0], W2.shape[1]))
    grad_b2= np.zeros((W2.shape[0], 1))
    
    c = ComputeCost(X=X, Y=Y, W1=W1, b1=b1, W2=W2, b2=b2, regularization_term=regularization_term)
    
    for i in range(b1.shape[0]):
        b1_try = np.copy(b1)
        b1_try[i, 0] += h
        c2 = ComputeCost(X=X, Y=Y, W1=W1, b1=b1_try, W2=W2, b2=b2, regularization_term=regularization_term)
        grad_b1[i,0] = (c2-c) / h

    for i in range(b2.shape[0]):
        b2_try= np.copy(b2)
        b2_try[i, 0] += h
        c2 = ComputeCost(X=X, Y=Y, W1=W1, b1=b1, W2=W2, b2=b2_try, regularization_term=regularization_term)
        grad_b2[i,0] = (c2-c) / h
        
    for i in range(W1.shape[0]):
        for j in range(W1.shape[1]):
            
            W1_try= np.copy(W1)
            W1_try[i,j] += h
            c2= ComputeCost(X=X, Y=Y, W1=W1_try, b1=b1, W2=W2, regularization_term=regularization_term)
            
            grad_W1[i,j] = (c2-c) / h

    for i in range(W2.shape[0]):
        for j in range(W2.shape[1]):
            W2_try = np.copy(W2)
            W2_try[i, j] += h
            c2 = ComputeCost(X=X, Y=Y, W1=W1, b1=b1, W2=W2_try, regularization_term=regularization_term)

            grad_W2[i, j] = (c2 - c) / h

    return W1, b1, W2, b2

In [206]:
def ComputeGradsNumSlow(X, Y, W1, b1, W2, b2, regularization_term, h=1e-5):
    """
    Computes gradient descent updates on a batch of data with numerical computations of great precision, thus slower computations.
    Contributed by Josephine Sullivan for educational purposes for the DD2424 Deep Learning in Data Science course.

    :param X: Input data
    :param Y: One-hot representation of the true labels of input data X
    :param W1: Weight matrix of the first layer
    :param b1: Bias vector of the first layer
    :param W2: Weight matrix of the second layer
    :param b2: Bias vector of the second layer
    :param regularization_term: Contribution of the regularization in the weight updates

    :return: Weight and bias updates of the first and second layer of our network computed with numerical computations with high precision.
    """

    grad_W1= np.zeros((W1.shape[0], W1.shape[1]))
    grad_b1= np.zeros((b1.shape[0], 1))
    grad_W2= np.zeros((W2.shape[0], W2.shape[1]))
    grad_b2= np.zeros((b2.shape[0], 1))
    
    for i in tqdm(range(b1.shape[0])):

        b1_try = np.copy(b1)
        b1_try[i,0] -= h
        c1 = ComputeCost(X=X, Y=Y, W1=W1, b1=b1_try, W2=W2, b2=b2, regularization_term=regularization_term)
        b1_try = np.copy(b1)
        b1_try[i,0] += h
        c2 = ComputeCost(X=X, Y=Y, W1=W1, b1=b1_try, W2=W2, b2=b2, regularization_term=regularization_term)
        grad_b1[i,0] = (c2-c1)/(2*h)

    for i in tqdm(range(b2.shape[0])):
        b2_try = np.copy(b2)
        b2_try[i, 0] -= h
        c1 = ComputeCost(X=X, Y=Y, W1=W1, b1=b1, W2=W2, b2=b2_try, regularization_term=regularization_term)

        b2_try = np.copy(b2)
        b2_try[i, 0] += h
        c2 = ComputeCost(X=X, Y=Y, W1=W1, b1=b1, W2=W2, b2=b2_try, regularization_term=regularization_term)
        grad_b2[i, 0] = (c2 - c1) / (2 * h)

    for i in tqdm(range(W1.shape[0])):
        for j in tqdm(range(W1.shape[1])):

            W1_try = np.copy(W1)
            W1_try[i, j] -= h
            c1 = ComputeCost(X=X, Y=Y, W1=W1_try, b1=b1, W2=W2, b2=b2, regularization_term=regularization_term)

            W1_try = np.copy(W1)
            W1_try[i, j] += h
            c2 = ComputeCost(X=X, Y=Y, W1=W1_try, b1=b1, W2=W2, b2=b2, regularization_term=regularization_term)

            grad_W1[i, j] = (c2 - c1) / (2 * h)

    for i in tqdm(range(W2.shape[0])):
        for j in tqdm(range(W2.shape[1])):

            W2_try = np.copy(W2)
            W2_try[i, j] -= h
            c1 = ComputeCost(X=X, Y=Y, W1=W1, b1=b1, W2=W2_try, b2=b2, regularization_term=regularization_term)

            W2_try = np.copy(W2)
            W2_try[i, j] += h
            c2 = ComputeCost(X=X, Y=Y, W1=W1, b1=b1, W2=W2_try, b2=b2, regularization_term=regularization_term)

            grad_W2[i, j] = (c2 - c1) / (2 * h)

    return W1, b1, W2, b2

In [23]:
def ComputeGradients(X, Y, W1, b1, W2, b2, regularization_term= 0):
    """
    Computes gradient descent updates on a batch of data

    :param X: Input data
    :param Y: One-hot representation of the true labels of input data X
    :param W1: Weight matrix of the first layer
    :param b1: Bias vector of the first layer
    :param W2: Weight matrix of the second layer
    :param b2: Bias vector of the second layer
    :param regularization_term: Contribution of the regularization in the weight updates

    :return: Weight and bias updates of the first and second layer of our network
    """

    # Evaluate the classifier to the batch
    p, h, s1 = EvaluateClassifier(X=X, W1=W1, b1=b1, W2=W2, b2=b2)

    # Back-propagate second layer at first

    g = p - Y
    grad_b2 = g.sum(axis=1).reshape(b2.shape)
    grad_W2 = np.dot(g, h.T)

    # Back-propagate the gradient vector g to the first layer
    g = np.dot(g.T, W2)
    ind = 1 * (s1 > 0)
    g = g.T * ind

    grad_b1 = np.sum(g, axis=1).reshape(b1.shape)
    grad_W1 = np.dot(g, X.T)

    grad_W1 /= X.shape[1]
    grad_b1 /= X.shape[1]
    grad_W2 /= X.shape[1]
    grad_b2 /= X.shape[1]

    # Add regularizers
    grad_W1 = grad_W1 + 2 * regularization_term * W1
    grad_W2 = grad_W2 +2 * regularization_term * W2

    return grad_W1, grad_b1, grad_W2, grad_b2

In [21]:
def check_similarity(gradW1, gradb1, gradW2, gradb2, gradW1_num, gradb1_num, gradW2_num, gradb2_num, threshold = 1e-4):

    """
    Compares the gradients of both the analytical and numerical method and prints out a message of result 
    or failure, depending on how close these gradients are between each other.
    
    :param gradW1: Gradient of W1, analytically computed
    :param gradb1: Gradient of b1, analytically computed
    :param gradW2: Gradient of W2, analytically computed
    :param gradb2: Gradient of b2, analytically computed
    :param gradW1_num: Gradient of W1, numerically computed
    :param gradb1_num: Gradient of b1, numerically computed
    :param gradW2_num: Gradient of W2, numerically computed
    :param gradb2_num: Gradient of b2, numerically computed

    :return: None
    """
    
    W1_abs = np.abs(gradW1 - gradW1_num)
    b1_abs = np.abs(gradb1 - gradb1_num)
    
    W2_abs = np.abs(gradW2 - gradW2_num)
    b2_abs = np.abs(gradb2 - gradb2_num)
    
    W1_nominator = np.average(W1_abs)
    b1_nominator = np.average(b1_abs)
    
    W2_nominator = np.average(W2_abs)
    b2_nominator = np.average(b2_abs)


    gradW1_abs = np.absolute(gradW1)
    gradW1_num_abs = np.absolute(gradW1_num)
    
    gradW2_abs = np.absolute(gradW2)
    gradW2_num_abs = np.absolute(gradW2_num)

    gradb1_abs = np.absolute(gradb1)
    gradb1_num_abs = np.absolute(gradb1)
    
    gradb2_abs = np.absolute(gradb2)
    gradb2_num_abs = np.absolute(gradb2)

    sum_W1 = gradW1_abs + gradW1_num_abs
    sum_W2 = gradW2_abs + gradW2_num_abs
    sum_b1 = gradb1_abs + gradb1_num_abs
    sum_b2 = gradb2_abs + gradb2_num_abs

    check_W1 = W1_nominator / np.amax(sum_W1)
    check_b1 = b1_nominator / np.amax(sum_b1)

    check_W2 = W2_nominator / np.amax(sum_W2)
    check_b2 = b2_nominator / np.amax(sum_b2)


    if check_W1 < threshold and check_b1 < threshold and check_W2 < threshold and check_b2 < threshold:
        print( "Success!!")
        print("Average error on weights of first layer= ", check_W1)
        print("Average error on bias of first layer=", check_b1)
        print("Average error on weights of second layer= ", check_W2)
        print("Average error on bias of second layer= ", check_b2)
    else:
        print("Failure")
        print("Average error on weights of first layer= ", check_W1)
        print("Average error on bias of first layer=", check_b1)
        print("Average error on weights of second layer= ", check_W2)
        print("Average error on bias of second layer= ", check_b2)


### ASSIGNMENT 2 EXERCISES

####  Exercise 1: Read in the data & initialize the parameters ofthe network

In [26]:
X_training_1, Y_training_1, y_training_1 = LoadBatch('../../cifar-10-batches-py/data_batch_1')
X_training_2, Y_training_2, y_training_2 = LoadBatch('../../cifar-10-batches-py/data_batch_2')
X_test, _, y_test = LoadBatch('../../cifar-10-batches-py/test_batch')

mean = np.mean(X_training_1)
X_training_1 -= mean
X_training_2 -= mean
X_test -= mean

In [27]:
W1, b1, W2, b2 = initialize_weights(d=X_training_1.shape[0], m=50, K=Y_training_1.shape[0])

#### Exercise 2: Compute the gradients for the network parameters

In [28]:
grad_W1, grad_b1, grad_W2, grad_b2 = ComputeGradients(X_training_1[:,0:2], Y_training_1[:,0:2], W1, b1, W2, b2)

Test no.1: Compare with numerically computed gradients

In [29]:
grad_W1_num = np.load('grad_W1_num.npy')
grad_b1_num = np.load('grad_b1_num.npy')
grad_W2_num = np.load('grad_W2_num.npy')
grad_b2_num = np.load('grad_b2_num.npy')

In [30]:
check_similarity(grad_W1, grad_b1, grad_W2, grad_b2, grad_W1_num, grad_b1_num, grad_W2_num, grad_b2_num)

Success!!
Average error on weights of first layer=  7.52283628908819e-09
Average error on bias of first layer= 4.602426280088195e-09
Average error on weights of second layer=  3.0747216543846533e-10
Average error on bias of second layer=  1.193638211533644e-11


#### Exercise 3: Add momentum to your update step